# Робота №4. Вивчення структури металів за допомогою функції радіального розподілу атомів

## Мета роботи
Дослідження параметрів структури металів в кристалічному та рідкому стані за допомогою визначення ФРР атомів моделюванням методом молекулярної динаміки.

## Об'єкт дослідження
Чисті метали за умов різних температур.

## Програма проведення експерименту
Створити кристал з ~1000-2000 атомів з періодичними граничними умовами. Матеріал: Pt.

In [1]:
%%writefile lab4.xm
read ../pot/Pt_xmd.eam

box 9 9 9

fill particle 4
1     0.0        0.0        0.0
1     0.5        0.0        0.5
1     0.5        0.5        0.0
1     0.0        0.5        0.5

fill align 8 8 8
fill orient 1 0 0  0 1 0  0 0 1
fill go

dtime 1e-14

select all #
mass 195.084

clamp $1
itemp $1
scale 3.9231

select index 14571
set add 1
select keep on

cmd 1000
#write file +lab2_pos_vel_$1.txt sel posvel
write pdb +lab4_$2.pdb
#WRITE file +lab4_$2.rdf RDF 160 2 18

Overwriting lab4.xm


In [2]:
Tmelt = 2041.4 # Kelvin
T = [0.1*Tmelt, 1.5*Tmelt]

In [3]:
for i, temp in enumerate(T):
    !del /f lab4_{i}.pdb
    #!del /f lab4_{i}.rdf
    !xmd lab4.xm {temp} {i} > lab4_{i}.log # temperature, n of experiment

In [53]:
16/0.1

160.0

In [45]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

def read_data(name):
    data_frame = pd.read_csv(name, delimiter='\s+', skipinitialspace=True, skiprows=4,
                             names=['skip1', 'HETATM', 'type', 'X', 'Y', 'Z'])[['X', 'Y', 'Z']]
    data_frame.dropna(inplace=True)
    return data_frame

data = [read_data(f'lab4_{i}.pdb') for i in range(2)]

In [49]:
centre = data[0].iloc[1456]
centre

X    15.609
Y    15.674
Z    15.765
Name: 1456, dtype: float64

In [42]:
data[0][['X', 'Y', 'Z']]

,X,Y,Z
0,0.037,35.275,0.060
1,1.957,0.049,1.916
2,1.896,1.970,35.229
3,35.303,1.948,1.934
4,0.036,0.019,3.882
5,1.876,0.013,5.843
6,2.001,1.981,3.907
7,0.035,1.947,5.946
8,35.305,35.271,7.852
9,2.014,35.256,9.819
